In [1]:
#Import Dependencies 
import json
import time
import pandas as pd
import csv
import matplotlib.pyplot as plt
import numpy as np
import calendar
import requests
import os
import pprint as pprint
#Import API Key
from config import gkey

#Read in Data
file = "crime_clean_2.csv"
crime_df = pd.read_csv(file)

#Preview Data
crime_df.head()

,Index,Date/Time,Date,Time,Status,Address,crime num,Crime Type,Committed/Attempted,Area Type,Sub Crime,Coordinates,Map,Lattitude,Longitude
0,167563,2010-08-13T12:40:00.000,8/13/2010,12:40:00,Arrest,4700 RALEIGH ST,2010-00380014,Narcotics,Committed,Highway/Road/Alley,Drugs/narcotics,"{'type': 'Point', 'coordinates': [-81.4421596,...",Mapped,28.5278265,"-81.4421596,"
1,167564,2011-03-14T20:11:00.000,3/14/2011,20:11:00,Closed,9200 AIRPORT BV,2011-00118002,Theft,Committed,Airport,All other larceny,"{'type': 'Point', 'coordinates': [-81.30852827...",Mapped,28.43180352,-81.30852827
2,167565,2017-12-25T12:45:00.000,12/25/2017,12:45:00,Closed,5300 MILLENIA BLVD,2017-01003353,Theft,Committed,Parking Lot - Surface,All other larceny,"{'type': 'Point', 'coordinates': [-81.44225326...",Mapped,28.47370534,-81.44225326
3,167566,2011-01-30T19:21:00.000,1/30/2011,19:21:00,Arrest,2700 E COLONIAL DR,2011-00047217,Theft,Committed,Specialty Store,Shoplifting,"{'type': 'Point', 'coordinates': [-81.35102496...",Mapped,28.55321394,-81.35102496
4,167567,2017-01-31T05:41:00.000,1/31/2017,5:41:00,Arrest,1 E PINE ST,2017-00041151,Theft,Committed,Restaurant/Fast Food,All other larceny,"{'type': 'Point', 'coordinates': [-81.37774666...",Mapped,8.541318854,-81.37774666


In [2]:
#Turn into DF and add zipcode column
df2 = {"zipcode":[]}
crime_df = crime_df.append(df2,ignore_index=True)

In [3]:
#Clean DataFrame
newdf= crime_df.sample(n=1000)
newdf= newdf.dropna(subset=['Lattitude', 'Longitude'])
newdf= newdf.drop(columns=['Date','Time','Committed/Attempted','Sub Crime','Coordinates','Map'])
newdf.head(1000)

,Index,Date/Time,Status,Address,crime num,Crime Type,Area Type,Lattitude,Longitude,zipcode
558,168121.0,2012-09-27T18:25:00.000,Closed,1000 BETHUNE DR,2012-00401607,Robbery,Highway/Road/Alley,28.52876767,-81.42010732,NaN
1212,168775.0,2011-11-29T18:29:00.000,Closed,5700 FOLKSTONE LN,2011-00524948,Burglary,Apartment/Condo,28.51808922,-81.30799681,NaN
1686,169249.0,2011-10-21T12:01:00.000,Closed,10100 TAVISTOCK RD,2011-00802454,Theft,Residence/Single,28.41324097,-81.24582898,NaN
61,167624.0,2015-05-12T10:32:00.000,Arrest,4300 E COLONIAL DR,2015-00194572,Fraud,Bank/Financial Institution,28.55344534,-81.33187925,NaN
1072,168635.0,2016-10-05T08:57:00.000,Closed,3800 N JOHN YOUNG PKWY,2016-00405363,Vehicle Theft,Auto Dealer/Car Lot,28.58449336,"-81.4167105,",NaN
2357,169920.0,2013-10-01T21:37:00.000,Closed,1300 JULIO LN,2013-00410666,Theft,Residence/Single,28.5264463,-81.31321909,NaN
1073,168636.0,2015-04-24T04:32:00.000,Closed,600 BARRY ST,2015-00167465,Vehicle Theft,Auto Dealer/Car Lot,28.55164038,-81.43403275,NaN
1199,168762.0,2010-08-17T06:28:00.000,Closed,1400 S KIRKMAN RD,2010-00386408,Vehicle Theft,Apartment/Condo,28.52503461,-81.45922298,NaN
2247,169810.0,2012-09-10T14:31:00.000,Closed,3900 W COLONIAL DR,2012-00376290,Burglary,Storage Unit,28.55267323,-81.42699228,NaN
133,167696.0,2015-12-25T21:22:00.000,Closed,6100 RALEIGH ST,2015-00528875,Burglary,Apartment/Condo,28.52790946,-81.46494688,NaN


In [4]:
#Use For loop & .itterrows to perform API calls requesting zipcodes of known coordinates
i=1
zips = []
apple =[]
for index, row in newdf.iterrows():
    lat = row['Lattitude']  
    long = row['Longitude']
    target_url = 'https://maps.googleapis.com/maps/api/geocode/json?latlng='+lat +','+long +'&result_type=postal_code'+'&key=' + gkey
    geo_data = requests.get(target_url).json()
    if len(geo_data["results"]) < 1:
        print("empty")
        apples ='00000'
        zips.append(apples)
    else:
        apples=geo_data['results'][0]['address_components'][0]['long_name']
        print(geo_data['results'][0]['address_components'][0]['long_name'])
        zips.append(apples)    
    
#Append requested zipcodes 
zips = newdf['zipcode']

32805
32822
32827
32803
empty


KeyboardInterrupt: 

In [ ]:
#Output the csv 
output = './finalfinaldata.csv'
newdf.to_csv(output, header=True, sep=',')

SyntaxError: invalid character in identifier (<ipython-input-7-df086baa9ad0>, line 3)